# Задание по курсу «Дискретная оптимизация», МФТИ, весна 2017

## Задача 2-2. Эвристика Кернигана—Лина

В этой задаче Вам предлагается добавить к локальному поиска в задаче о сбалансированном разбиении графа эвристику Кернигана—Лина, когда мы, «застряв» в локальном минимуме, тем не менее пытаемся сделать несколько шагов из него, даже если они приводят к временному ухудшению. Надежда здесь на то, что после ухудшения может наступить заметное улучшение результата: нам удастся выпрыгнуть из локального оптимума. Мы рассматриваем безвесовый вариант задачи о разбиении с параметром балансировки $\alpha=\frac{1}{2}$:

**Даны:**
* $G=(V,E)$ — граф без весов на рёбрах

**Найти:**
* Разбиение $V=V'\sqcup V''$, такое, что $V'=\lfloor |V|/2 \rfloor$ и число рёбер между $V'$ и $V''$ минимально возможное.

Сделайте следующее:
* Скачайте файл [`partition-instances.zip`](https://github.com/dainiak/discrete-optimization-course/raw/master/partition-instances.zip) и разархивируйте из него файлы со входами задачи.
* Для каждого из графов найдите локальным поиском с эвристикой Кернигана—Лина локально минимальное (по количеству рёбер между частями) разбиение вершин графа на две части, мощности которых отличаются не более чем на единицу. 
* Реализуйте функцию `variable_depth_local_search`; она должна принимать на вход граф в формате, предоставляемом функцией `read_instance`, и возвращать найденное разбиение как множество вершин, лежащих в одной любой из двух компонент разбиения. Ваш локальный поиск должен начинаться с того разбиение, которое уже находится в переменной `starting_point`.
* Подберите для каждого из четырёх входных графов глубину поиска так, чтобы он работал не более 60 секунд на Вашем компьютере, и сохраните информацию о подобранных параметрах и любые свои интересные наблюдения в отдельную ячейку настоящего ipynb-файла.

In [117]:
def read_instance(filename):
    with open(filename, 'r') as file:
        n_vertices = int(file.readline().strip().split()[0])
        vertices, edges = set(range(1, n_vertices + 1)), set()
        for u in range(1, n_vertices + 1):
            for v in map(int, file.readline().strip().split()):
                edges.add((u,v))
        return (vertices, edges)

In [232]:
import time
def variable_depth_local_search(graph):
    def get_weight(left):
        ans = 0
        for edge in edges:
            a, b = edge
            if a in left and not b in left:
                ans += 1
        return ans

    def move(ch1, a, b):
        if (a in ch1[0]):
            ch1[0].discard(a)
        else:
            ch1[1].add(a)
        if (b in ch1[1]):
            ch1[1].discard(b)
        else:
            ch1[0].add(b)
                
    time_start = time.monotonic()
    cnt = 0
    x = set()
    
    starting_point = set(range(1, len(graph[0]) // 2 + 1))
    vertices, edges = graph
    n = len(vertices)
    G = [[] for i in range(1 + n)]
    for edge in edges:
        a, b = edge
        G[a].append(b)
    left = list(starting_point)
    bestleft = left[:]
    right = list(range(len(graph[0]) // 2 + 1, len(graph[0]) + 1))
    rec = 200 #длина прыжка
    if n > 10000:
        rec = 20
    opt = get_weight(starting_point)
    r = 0
    parent = (-1, -1)
    cur = opt
    before = [] #будем во время прыжка хранить посещенные состояния
    ch = [set(), set()] #добавленные в левую и правую доли, описывают разбиение с момента начала длинного прыжка
    cost_left = [0 for i in range(n+1)]
    cost_right = [0 for i in range(n+1)]
    for a in left:
        for b in G[a]:
            cost_left[b] += 1
    for b in right:
        for a in G[b]:
            cost_right[a] += 1
    while (True):
        cnt += 1
        time_elapsed = time.monotonic()-time_start
        if time_elapsed > 45:
            break
        if rec == r:
            break
        ch1 = [set(ch[0]), set(ch[1])]
        before += [ch1]
        best = -1
        bestpair = (-1, -1)
        left.sort(key=lambda a:cost_left[a] - cost_right[a])
        right.sort(key=lambda a:-cost_left[a] + cost_right[a])
        quit = False
        for i in range(1, len(left)):
            if quit:
                break
            for j in range(1, len(right)):
                a = left[i]
                b = right[j]
                now = cur
                now += cost_left[a] - cost_right[a]
                now += -cost_left[b] + cost_right[b]
                oldnow = now
                if best != -1 and now >= min(opt, best): #дальше все варианты будут заведомо хуже, 
                    #в силу сортировки лучшие варианты при меньших i и j
                    quit = True
                    break
                if (a, b) in edges:
                    now += 2
                ch1 = [set(ch[0]), set(ch[1])]
                move(ch1, a, b)
                if ch1 in before: #уже были там в этой ветке
                    continue
                if (best == -1 or now < best):
                    best = now
                    bestpair = (i, j)
        if best == -1:
            break
        i, j = bestpair
        a, b = left[i], right[j]
        left[i], right[j] = right[j], left[i]
        cur = best
        for bb in G[a]:
            cost_left[bb] -= 1
            cost_right[bb] += 1
        for aa in G[b]:
            cost_left[aa] += 1
            cost_right[aa] -= 1
        if best < opt:
            opt = best
            bestleft = left[::-1]
            before = []
            ch = [set(), set()]
            r = 0
        else:
            r += 1
            move(ch, a, b)
    return set(bestleft)


In [233]:
import time

def get_quality(graph, partition_part):
    if not (partition_part <= graph[0]) or abs(len(partition_part) - len(graph[0]) / 2) > 0.6:
        return -1
    other_part = set(graph[0]) - partition_part
    return sum(1 for edge in graph[1] if set(edge) <= partition_part or set(edge) <= other_part )

def run_all():
    filenames = ['add20.graph', 'cti.graph', 't60k.graph', 'm14b.graph']
    for filename in filenames:
        instance = read_instance(filename)
        print('Solving instance {}…'.format(filename), end='')
        time_start = time.monotonic()
        quality = get_quality(instance, variable_depth_local_search(instance))
        time_elapsed = time.monotonic()-time_start
        print(' done in {:.2} seconds with quality {}'.format(time_elapsed, quality))

In [234]:
run_all()

Solving instance add20.graph… done in 0.36 seconds with quality 12142
Solving instance cti.graph… done in 0.42 seconds with quality 94072
Solving instance t60k.graph… done in 1.1 seconds with quality 178244
Solving instance m14b.graph… done in 5.1e+01 seconds with quality 2908144


## Выводы
(Здесь опишите свои наблюдения и подобранные параметры для каждогр из четырёх входных графов.)

In [ ]:
Работает на последнем тесте очень долго, несмотря на оптимизации. Для более маленьких можно брать больше длину прыжка.
На последнем пришлось отсекать по времени.